In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from matplotlib.colors import PowerNorm, LogNorm
from scipy.spatial.distance import pdist
from matplotlib import animation

In [ ]:
from IPython.display import HTML

In [ ]:
def get_params_posits(fname):
    fname = Path(fname)
    if not fname.exists():
        raise RuntimeError("File not found")
    if not fname.name.split('.')[-1] == 'analysis' and fname.name.split('.')[-2] == 'msd':
        raise RuntimeError("File must be a MSD analysis file with the format "
                           "'<file name>.msd.analysis'")
    params = pd.read_csv(fname, delim_whitespace=True, nrows=1)
    posits = pd.read_csv(fname, index_col=0, delim_whitespace=True, skiprows=2)    
    n_fils = params.n_filaments[0]
    fil_labels = [i for sub in [["fil{:04d}".format(i)] * 6 for i in range(n_fils)] for i in sub]
    arrays = [fil_labels, ["x", "y", "z", "ux", "uy", "uz"] * n_fils]
    columns = pd.MultiIndex.from_arrays(arrays, names=["filament", "coord"])
    posits.columns = columns
    return params, posits

def get_msd_vcf_from_posits(posits):
    """Get MSD and VCF from posits"""
    time_len = posits.shape[0]//4
    dr2 = np.zeros(time_len * params.n_filaments[0])
    du2 = np.zeros(time_len * params.n_filaments[0])
    start_times = range(0, posits.shape[0] - time_len, time_len//4)
    for start in start_times:
        pos = (posits.iloc[start:start+time_len] - posits.iloc[start]).stack('filament').iloc[:, 3:]
        u = (posits.iloc[start:start+time_len] - posits.iloc[start]).stack('filament').iloc[:, :3]
        dr2 = dr2 + np.sum(pos.values**2, axis=1)
        du2 = du2 + np.sum(u.values**2, axis=1)
    dr2 /= len(start_times)
    du2 /= len(start_times)
    pos = pd.DataFrame(dr2, columns=['dr2'], index=pos.index).unstack('filament')
    u = pd.DataFrame(du2, columns=['du2'], index=u.index).unstack('filament')
    pos.columns = list(range(pos.shape[1]))
    u.columns = list(range(u.shape[1]))
    pos_mean = pos.mean(axis=1)
    pos_stderr = pos.std(axis=1)/np.sqrt(pos.shape[1])
    u_mean = u.mean(axis=1)
    u_stderr = u.std(axis=1)/np.sqrt(u.shape[1])
    return (pos_mean, pos_stderr), (u_mean, u_stderr)

def run_msd_analysis(fname, late_time_percentage, show_plots=False, save_plots=False,
                     dist_lag_times=[10, 100, 1000], dist_xlims=None):
    params, posits = get_params_posits(fname)
    assert (late_time_percentage > 0 and late_time_percentage <= 1), (
        "Late time percentage must be a value between 0 and 1"
    )
    posit_start = int(late_time_percentage * posits.shape[0])
    msd, vcf = get_msd_vcf_from_posits(posits.iloc[posit_start:, :])
    if save_plots or show_plots:
        fig, ax = plt.subplots(1, 2, figsize=(14, 6))
        time = msd[0].index - msd[0].index[0]
        ax[0].plot(time, msd[0], label='MSD')
        ax[0].fill_between(time, msd[0]-msd[1], msd[0]+msd[1], alpha=0.5, label='s.e.m.')
        ax[1].plot(time, vcf[0], label='VCF')
        ax[1].fill_between(time, vcf[0]-vcf[1], vcf[0]+vcf[1], alpha=0.5, label='s.e.m.')
        ax[0].legend(loc='upper left', fontsize=15)
        ax[1].legend(loc='upper left', fontsize=15)
        ax[0].set_xlabel(r'$\tau$', fontsize=18)
        ax[1].set_xlabel(r'$\tau$', fontsize=18)
        ax[0].set_ylabel(r'$\langle (\mathbf{r}(0) - \mathbf{r}(t))^2 \rangle$', fontsize=18)
        ax[1].set_ylabel(r'$\langle (\mathbf{u}(0) - \mathbf{u}(t))^2 \rangle$', fontsize=18)
        fig.tight_layout()
        ax[0].tick_params(labelsize=15)
        ax[1].tick_params(labelsize=15)
        if show_plots:
            plt.show()
        if save_plots:
            fig.savefig(Path(fname.parent, fname.name + '.png'), dpi=200, bbox_inches='tight')
        plt.close()
        plot_lag_time_distributions(fname, posits, lag_times=dist_lag_times, save=save_plots,
                                    show=show_plots, dist_xlims=dist_xlims)
    return msd, vcf

def plot_lag_time_distributions(fname, posits, lag_times=[10, 100, 1000], save=False, 
                                show=True, dist_xlims=None):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    plot_lag_time_dists(posits, ax[0], lag_times=lag_times, dimension='x', xlims=dist_xlims)
    plot_lag_time_dists(posits, ax[1], lag_times=lag_times, dimension='y', xlims=dist_xlims)
    fig.tight_layout()
    if show:
        plt.show()
    if save:
        fig.savefig(Path(fname.parent, fname.name + '_lag_time_dists.png'), dpi=300, bbox_inches='tight')
    plt.close()

def plot_lag_time_dists(posits, ax, lag_times=[10, 100, 1000], dimension='x', xlims=None):
    if dimension == 'x':
        index = 3
    elif dimension == 'y':
        index = 4
    else:
        raise RuntimeError("Dimension should be 'x' or 'y'")
    if xlims is not None:
        bins = np.linspace(xlims[0], xlims[1], 36)
    else:
        bins = 35
    for T in lag_times:
        ax.hist(posits.diff(periods=T).dropna().stack('filament').iloc[:, index].values,
                bins=bins, density=True, histtype='step', linewidth=2, label=T)
    ax.set_xlabel(r'$\Delta$' + dimension + r'($\tau$)', fontsize=18)
    ax.set_ylabel(r'P($\Delta$' + dimension + r'($\tau$))', fontsize=18)
    legend = ax.legend(loc='upper right', title=r'$\tau$', fontsize=15)
    ax.tick_params(labelsize=15)
    legend.get_title().set_fontsize('18')

def run_cluster_msd_analysis(fname, lifetime_min, show_plots=True, save_plots=False, drop_nans=True):
    df = pd.read_csv(fname, delim_whitespace=True)
    lifetimes = df.groupby('cluster_label').count().sort_values(by='time', ascending=False).time
    long_lived_labels = lifetimes[lifetimes > lifetime_min].index
    dr2_df = None
    for label in long_lived_labels:
        dr2_df = get_msd_vcf_from_cluster(df.loc[df['cluster_label'] == label], dr2_df)
    if drop_nans:
        dr2_df = dr2_df.dropna()
    dr2_mean = dr2_df.mean(axis=1)
    dr2_std = dr2_df.std(axis=1)/np.sqrt(dr2_df.shape[1])

    if save_plots or show_plots:
        fig = plt.figure(figsize=(6, 4))
        ax = fig.gca()
        time = dr2_df.index
        ax.plot(time, dr2_mean, label='MSD')
        ax.fill_between(time, dr2_mean-dr2_std, dr2_mean+dr2_std, alpha=0.5, label='s.e.m.')
        ax.legend(loc='upper left', fontsize=15)
        ax.set_xlabel(r'$\tau$', fontsize=18)
        ax.set_ylabel(r'$\langle (\mathbf{r}(0) - \mathbf{r}(t))^2 \rangle$', fontsize=18)
        fig.tight_layout()
        ax.tick_params(labelsize=15)
        ax.set_title(r"Filament cluster MSD, $N$ = {}, $n$ = {}".format(long_lived_labels.shape[0], 
                                                                        dr2_df.shape[1]), fontsize=20)
        if show_plots:
            plt.show()
        if save_plots:
            fig.savefig(Path(fname.parent, fname.name + '.msd.png'), dpi=200, bbox_inches='tight')
        plt.close()
    return dr2_df
            
def get_msd_vcf_from_cluster(cluster_df, dr2_df = None):
    """Get MSD and VCF from cluster posits"""
    assert cluster_df.cluster_label.nunique() == 1, "Found multiple cluster labels in cluster dataframe"
    time_len = cluster_df.shape[0]//4
    if dr2_df is None:
        dr2 = np.zeros(time_len)
    else:
        dr2 = np.zeros(dr2_df.shape[0])
        dr2[time_len:] = np.nan
    start_times = range(0, cluster_df.shape[0] - time_len, time_len//4)
    posits = cluster_df.iloc[:, 3:5]
    for i, start in enumerate(start_times):
        pos = (posits.iloc[start:start+time_len] - posits.iloc[start])
        dr2[:time_len] = np.sum(pos.values**2, axis=1)
        if dr2_df is None:
            dr2_df = pd.DataFrame(dr2, columns=[cluster_df.cluster_label.iloc[0]],
                                  index=cluster_df.time.iloc[:time_len] - cluster_df.time.iloc[0])
        else:
            dr2_df['{}.{}'.format(cluster_df.cluster_label.iloc[0], i)] = dr2
    return dr2_df

In [ ]:
lag_times = [25, 100, 400, 800, 1600, 3200]

In [ ]:
fname = Path("ic_nodr_v020_filament_mt.msd.analysis")
msd, vcf = run_msd_analysis(fname, late_time_percentage=0.1,
                            save_plots=True, show_plots=True, 
                            dist_lag_times=lag_times,
                            dist_xlims = (-15, 15))

In [ ]:
fname = Path("ic_v020_filament_mt.msd.analysis")
msd, vcf = run_msd_analysis(fname, late_time_percentage=0.4,
                            save_plots=True, show_plots=True,
                            dist_lag_times=lag_times,
                            dist_xlims=(-15, 15))

In [ ]:
fname = Path("ic_nodr_v037_filament_mt.msd.analysis")
msd, vcf = run_msd_analysis(fname, late_time_percentage=0.2,
                            save_plots=True, show_plots=True,
                            dist_lag_times=lag_times,
                            dist_xlims=None)

In [ ]:
fname = Path("ic_v037_filament_mt.msd.analysis")
msd, vcf = run_msd_analysis(fname, late_time_percentage=0.4,
                            save_plots=True, show_plots=True,
                            dist_lag_times=lag_times,
                            dist_xlims=(-30, 30))

In [ ]:
params, posits = get_params_posits(Path("ic_v037_filament_mt.msd.analysis"))

In [ ]:
posits.head()

In [ ]:
posits = posits.dropna()

In [ ]:
posits = posits.iloc[posits.shape[0]//4:]

In [ ]:
posits_only = posits.stack('filament').iloc[:, 3:5].unstack('filament').reorder_levels(
    ['filament', 'coord'], axis=1).sort_index(axis=1)

In [ ]:
def get_lag_diff(posits, lag_time):
    return posits.diff(periods=lag_time).dropna().iloc[1:].stack('filament')

In [ ]:
lag_times = np.exp(np.linspace(0, 9, 40))
lag_times[0] = 0
lag_times = np.unique([int(t) for t in lag_times])
hists = np.array([np.histogram2d(diff.x, diff.y, bins=np.linspace(-20, 20, 100), density=True)[0]
                  for diff in
                  [get_lag_diff(posits_only, int(T)) for T in lag_times]])

In [ ]:
font = {'family': 'DejaVu Sans Mono',
        'color':  'black',
        'weight': 'normal',
        'size': 16,
        }
fig = plt.figure(figsize=(8, 8))
ax = fig.gca()
cax = ax.imshow(hists[0], cmap=plt.cm.coolwarm, vmin=0, vmax=1, norm=LogNorm(), animated=True, origin='lower')
txt = ax.text(0.7, 0.9, r"$\tau$ = {:06.02f}".format(0), fontdict=font,transform=ax.transAxes, 
              bbox=dict(facecolor='white', alpha=0.8))
cbar = fig.colorbar(cax, shrink=0.8, )
ax.tick_params(labelsize=13)
ax.set_xticks(np.linspace(0, 98, 5))
ax.set_xticklabels([str(i) for i in np.linspace(-20, 20, 5)])
ax.set_yticks(np.linspace(0, 98, 5))
ax.set_yticklabels([str(i) for i in np.linspace(-20, 20, 5)])
ax.set_xlabel('x', fontsize=18)
ax.set_ylabel('y', fontsize=18)
ax.set_title('2D Autocorrelation Distribution Function', fontsize=20)
cbar.ax.tick_params(labelsize=14)
cbar.ax.set_title(r'$\rho$', fontsize=20)
times = posits.index - posits.index[0]
def animate(i):
    cax.set_array(hists[i]);
    txt.set_text(r"$\tau$ = {:06.02f}".format(times[int(lag_times[i])]))
ani = animation.FuncAnimation(
    fig, animate, interval=100, frames=range(len(hists)))
ani.save("ic_v037_vh_self.mp4")

In [ ]:
HTML(ani.to_html5_video())

In [ ]:
params, posits = get_params_posits(fname)

In [ ]:
pd.DataFrame().unstack()

In [ ]:
posits.head().stack('filament').iloc[:, 3:5].unstack('time').reorder_levels(
    ['time', 'coord'], axis=1).sort_index(axis=1)

In [ ]:
posits_only = posits.stack('filament').iloc[:, 3:5].unstack('filament').reorder_levels(
    ['filament', 'coord'], axis=1).sort_index(axis=1)
posits_by_time = posits_only.stack('filament').unstack('time').reorder_levels(
    ['time', 'coord'], axis=1).sort_index(axis=1)

In [ ]:
xperiodic = 50
def euclidean_pbc_1d(u, v):
    #x = u[0] - v[0]
    #if x < 0:
    #    x = (x / xperiodic - x // xperiodic - 1) * xperiodic
    #else:
    #    x = (x / xperiodic - x // xperiodic) * xperiodic
    #return x
    return u[0] - v[0]

In [ ]:
def my_func(array, xperiodic):
    N = array.shape[0]
    result = np.zeros(int(N*(N-1)/2))
    k = 0
    for i in range(N-1):
        for j in range(i+1, N):
            x = array[i] - array[j]
            if (x < 0):
                result[k] = (x / xperiodic - x // xperiodic - 1) * xperiodic
            else:
                result[k] = (x / xperiodic - x // xperiodic) * xperiodic
            k+=1

In [ ]:
%%timeit -n1 -r1
bins = np.linspace(-15, 15, 100)
lag_times = range(10, 10000, 100)

x0 = pdist(posits_by_time.iloc[:, 0:2], euclidean_pbc_1d)
y0 = pdist(posits_by_time.iloc[:, 1:3], euclidean_pbc_1d)
mask = (abs(x0) > 0)
hist = np.histogram2d(x0[mask], y0[mask], bins=bins)[0]
for i in lag_times:
    x0 = pdist(posits_by_time.iloc[:, i:i+2], euclidean_pbc_1d)
    y0 = pdist(posits_by_time.iloc[:, i+1:i+3], euclidean_pbc_1d)
    x0[x0 < 0] = (x0[x0 < 0] / xperiodic - x0[x0 < 0] // xperiodic) * xperiodic
    x0[x0 > 0] = (x0[x0 > 0] / xperiodic - x0[x0 > 0] // xperiodic) * xperiodic
    y0[y0 < 0] = (y0[y0 < 0] / xperiodic - y0[y0 < 0] // xperiodic) * xperiodic
    y0[y0 > 0] = (y0[y0 > 0] / xperiodic - y0[y0 > 0] // xperiodic) * xperiodic
    mask = (abs(x0) > 0)
    hist += np.histogram2d(x0[mask], y0[mask], bins=bins)[0]
hist /= len(lag_times)+1

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.gca()
cax = ax.imshow(hist, cmap=plt.cm.coolwarm, vmin=0, vmax=1, norm=PowerNorm(1), animated=True, origin='lower')
cbar = fig.colorbar(cax, shrink=0.8, )
ax.tick_params(labelsize=13)
ax.set_xticks(np.linspace(0, 98, 5))
ax.set_xticklabels([str(i) for i in np.linspace(-20, 20, 5)])
ax.set_yticks(np.linspace(0, 98, 5))
ax.set_yticklabels([str(i) for i in np.linspace(-20, 20, 5)])
ax.set_xlabel('x', fontsize=18)
ax.set_ylabel('y', fontsize=18)
cbar.ax.tick_params(labelsize=14)
cbar.ax.set_title(r'$\rho$', fontsize=20)
ax.set_title('2D Pair Distribution Function', fontsize=18)
#fig.savefig('ic_v037_pdf.png', dpi=300)
plt.show()

In [ ]:
x0[abs(x0)>0].shape

In [ ]:
y0[abs(y0)>0].shape

In [ ]:
params

In [ ]:
x0[x0>0].shape

In [ ]:
x0.shape

In [ ]:
fname = Path('ic_v037_filament_mt.van_hove_distinct.analysis')
vh_params = pd.read_csv(fname, delim_whitespace=True, nrows=1)
lag_times = pd.read_csv(fname, delim_whitespace=True, skiprows=3, header=None, nrows=1)
n_samples = pd.read_csv(fname, delim_whitespace=True, skiprows=5, header=None, nrows=1)
data = pd.read_csv(fname, delim_whitespace=True, skiprows=6, header=None)
n_bins_1d = vh_params.n_bins_1d.iloc[0]
n_frames = vh_params.n_frames.iloc[0]
lag_times = lag_times.iloc[0].values
n_samples = n_samples.iloc[0].values
n_fil = 232
data = data.values
data_distinct = []
for i in range(n_frames):
    data_distinct.append(data[i*n_bins_1d:i*n_bins_1d+n_bins_1d])
data_distinct = np.array(data_distinct)

In [ ]:
fname = Path('ic_v037_filament_mt.van_hove_self.analysis')
vh_params = pd.read_csv(fname, delim_whitespace=True, nrows=1)
lag_times = pd.read_csv(fname, delim_whitespace=True, skiprows=3, header=None, nrows=1)
n_samples = pd.read_csv(fname, delim_whitespace=True, skiprows=5, header=None, nrows=1)
data = pd.read_csv(fname, delim_whitespace=True, skiprows=6, header=None)
n_bins_1d = vh_params.n_bins_1d.iloc[0]
n_frames = vh_params.n_frames.iloc[0]
lag_times = lag_times.iloc[0].values
n_samples = n_samples.iloc[0].values
n_fil = 232
data = data.values
data_self = []
for i in range(n_frames):
    data_self.append(data[i*n_bins_1d:i*n_bins_1d+n_bins_1d])
data_self = np.array(data_self)

In [ ]:
data = data_self + data_distinct
data_F = []
for i in range(n_frames):
    data_F.append(np.fft.fftshift(np.fft.fft2(data[i])))
data_F = np.absolute(data_F)

In [ ]:
data_S = np.fft.fft(data_F, axis=0)
data_S = np.absolute(data_S)

In [ ]:
data=data_S
vmax = (1.05*dat.max() if dat.max() < 0.9 else 1)
vmax = 5

In [ ]:
offset = 0
font = {'family': 'DejaVu Sans Mono',
        'color':  'black',
        'weight': 'normal',
        'size': 16,
        }
fig = plt.figure(figsize=(8, 8))
ax = fig.gca()
cax = ax.imshow(data[0, offset:n_bins_1d-offset, offset:n_bins_1d-offset],
                cmap=plt.cm.coolwarm, vmin=0, vmax=vmax, norm=PowerNorm(1),
                animated=True, origin='lower')
txt = ax.text(0.7, 0.9, r"$\tau$ = {:06.02f}".format(0),
              fontdict=font,transform=ax.transAxes,
              bbox=dict(facecolor='white', alpha=0.8))
cbar = fig.colorbar(cax, shrink=0.8, )
ax.tick_params(labelsize=13)
ticks = np.linspace(-(n_bins_1d-2*offset-1)/4, (n_bins_1d-2*offset-1)/4, 5)
ax.set_xticks(np.linspace(0, n_bins_1d-2*offset-1, 5))
ax.set_xticklabels([str(i) for i in ticks])
ax.set_yticks(np.linspace(0, n_bins_1d-2*offset-1, 5))
ax.set_yticklabels([str(i) for i in ticks])
ax.set_xlabel('x', fontsize=18)
ax.set_ylabel('y', fontsize=18)
ax.set_title('2D Autocorrelation Distribution Function', fontsize=20)
cbar.ax.tick_params(labelsize=14)
cbar.ax.set_title(r'$\rho$', fontsize=20)
times = posits.index - posits.index[0]
def animate(i):
    cax.set_array(data[i, offset:n_bins_1d-offset, offset:n_bins_1d-offset])
    txt.set_text(r"$\tau$ = {:06.02f}".format(lag_times[i]))
ani = animation.FuncAnimation(
    fig, animate, interval=100, frames=n_frames-2)
#ani.save("ic_v037_vh_self.mp4")

In [ ]:
HTML(ani.to_html5_video())